# Notebook completo

Neste arquivo, encontra-se o modelo escolhido para compor a entrega final do projeto. O algoritmo apresentado foi selecionado com base em seus resultados, que se destacaram entre todos os modelos testados, considerando nossas métricas de avaliação (accuracy, precision, recall e F1-score).

Vale ressaltar que o modelo presente neste arquivo é o que está integrado em nossa API e realiza a classificação de sentimentos. O arquivo está dividido nas seguintes etapas:

- Importação de bibliotecas
- Pré-processamento dos dados
- Vetorização dos dados
- Modelo de Machine Learning
- Exportação do modelo

O conteúdo abrange todos os elementos essenciais para realização de testes e treino do modelo, também realiza todas as etapas necessárias para o modelo, como pré-processamento, entre outras etapas.

### Importação das bibliotecas

A importação de bibliotecas é o primeiro passo em um projeto de machine learning. Consiste em trazer para o ambiente de trabalho os pacotes e módulos necessários que contêm funções e ferramentas pré-construídas. Essas bibliotecas facilitam a manipulação de dados, a construção de modelos, a análise estatística e a visualização de resultados.

In [ ]:
import pandas as pd
import nltk
import re
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
import string
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from spellchecker import SpellChecker
from nltk import pos_tag
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import cohen_kappa_score

# Baixar os recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load("en_core_web_sm")


### Pré-Processamento dos dados

O pré-processamento dos dados envolve a preparação dos dados brutos para que possam ser usados de maneira eficaz pelo modelo de machine learning. O objetivo é melhorar a qualidade dos dados, utilizando de técnicas como, remoção de stopwords, tokenização, entre outras, o que, por sua vez, melhora o desempenho do modelo.

##### Importação dos dados

In [ ]:
# Obter o diretório atual (funciona no Jupyter Notebook)
current_dir = os.getcwd()
csv_path = os.path.join(current_dir, 'dados.csv')
# Carregar o arquivo CSV
df = pd.read_csv(csv_path)
df.head()

##### Tokenização

A tokenização é o processo de dividir um texto em unidades menores chamadas tokens. Esses tokens podem ser palavras individuais, partes de palavras ou até mesmo caracteres, dependendo do nível de granularidade desejado. A tokenização é uma etapa fundamental no processamento de linguagem natural (PLN), sendo essencial para a preparação, análise e manipulação de texto em uma variedade de aplicações, incluindo análise de sentimento, classificação de texto e tradução automática. Ao dividir o texto em tokens, os dados tornam-se estruturados e adequados para análise, facilitando a extração de informações e a modelagem de soluções de PLN.

In [ ]:
def tokenize_text(text):
    """
    Tokeniza o DataFrame em uma lista de tokens.

    Args:
    text (DataFrame): Comentários a serem tokenizados.

    Returns:
    list: Lista de tokens.
    """
    tokens = word_tokenize(text.lower())
    return tokens

def filter_empty_tokens(tokens):
    """
    Remove listas vazias ou com espaços vazios.

    Args:
    tokens (list): Lista de tokens.

    Returns:
    list: Lista de tokens não vazios.
    """
    return [token for token in tokens if token.strip()]

##### Lemmatização

A lemmatização é o processo de reduzir palavras a sua forma base ou lema, considerando o contexto e a morfologia da língua. Essa técnica é importante em PLN para tratar diferentes formas de uma palavra como iguais, como "corre" e "correu" ambas reduzidas a "correr". Isso simplifica o processamento de texto e melhora a análise em tarefas como recuperação de informações e análise de sentimento.

In [ ]:
def get_wordnet_pos(treebank_tag):
    """
    Retorna o tag correspondente do WordNet para o tag do Treebank do Penn.

    Args:
        treebank_tag (str): O tag do Treebank do Penn que precisa ser convertido.

    Returns:
        str: O tag correspondente do WordNet (ADJ, VERB, NOUN, ADV).
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 

def lemmatize_tokens_with_pos(tokens):
    """
    Lemmatiza uma lista de tokens baseando-se em sua parte do discurso.

    Args:
    tokens (list of str): Tokens a serem lematizados.

    Returns:
    list: Lista de lemas das palavras.
    """
    # Criar um Doc do spaCy a partir dos tokens
    doc = nlp(" ".join(tokens))

    # Lemmatiza usando o spaCy
    lemmas = [token.lemma_ for token in doc]
    return lemmas

##### Remoção de pontuação

A remoção de pontuação é um processo utilizado no pré-processamento de texto que visa eliminar caracteres de pontuação, como vírgulas, pontos e pontos de exclamação, de um texto. Isso é feito para limpar o texto e reduzir a dimensionalidade dos dados, facilitando a análise e a modelagem. Ao remover a pontuação, os tokens resultantes contêm apenas palavras ou partes de palavras, tornando-os mais adequados para tarefas de processamento de linguagem natural.

In [ ]:
def remove_punctuation_from_tokens(tokens):
    """
    Remove pontuações de uma lista de tokens e exclui os tokens que consistem exclusivamente de caracteres de pontuação.

    Args:
    tokens (list): Lista de tokens a serem processados.

    Returns:
    list: Lista de tokens sem pontuações.
    """
    # Regex para identificar pontuações
    regex_punctuation = re.compile('[%s]' % re.escape(string.punctuation))

    # Remove pontuações de cada token e filtra tokens que ficaram vazios ou são apenas pontuações
    tokens_no_punct = [regex_punctuation.sub('', token) for token in tokens]
    tokens_no_punct = [token for token in tokens_no_punct if token.strip() != '']

    return tokens_no_punct

##### Remoção de números

A remoção de números é um passo comum no pré-processamento de texto que consiste em eliminar todos os caracteres numéricos de um texto. Isso é feito para limpar o texto de informações numéricas que podem não ser relevantes para a análise ou para garantir que as palavras sejam tratadas de maneira uniforme durante a tokenização. Ao remover números, os tokens resultantes contêm apenas palavras e outros caracteres não numéricos, simplificando o texto para análise e modelagem.

In [ ]:
def remove_numbers_from_tokens(tokens):
    """
    Remove todos os dígitos de uma lista de tokens e remove os tokens que consistem exclusivamente de números.

    Args:
    tokens (list): Lista de tokens a serem processados.

    Returns:
    list: Lista de tokens sem números.
    """
    # Remover dígitos de cada token e depois filtrar os tokens que são apenas números ou ficaram vazios
    tokens_no_numbers = [re.sub(r'\d+', '', token) for token in tokens]
    tokens_no_numbers = [token for token in tokens_no_numbers if token.strip() != '']
    return tokens_no_numbers

##### Remoção de Stop Words

Stopwords são palavras que são frequentemente removidas durante o pré-processamento de texto em tarefas de Processamento de Linguagem Natural (PLN). Essas palavras são geralmente as mais comuns em um idioma, como ‘é’, ‘em’, ‘um’, ‘e’ em português, ou ‘is’, ‘in’, ‘a’, ‘and’ em inglês, e tendem a aparecer em quase todos os documentos de um corpus.

A remoção de stopwords é uma prática comum porque essas palavras, embora muito frequentes, geralmente não carregam muito significado e podem adicionar ruído aos dados. Além disso, removendo-as, podemos reduzir o tamanho do nosso vocabulário e, consequentemente, o espaço de recursos, tornando nossos modelos de PLN mais eficientes.

In [ ]:
def remove_stopwords_preserve_adverbs_conjunctions(tokens):
    """
    Remove stopwords de uma lista de tokens enquanto preserva todos os advérbios e conjunções, 
    que são cruciais para manter o fluxo lógico e a estrutura das frases.

    Args:
        tokens (list): Lista de tokens a serem processados.

    Returns:
        list: Lista de tokens sem stopwords, com todos os advérbios e conjunções preservados.
        
    """
    if not isinstance(tokens, list):
        raise TypeError("Input tokens must be a list")
    
    # Loading stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.add('uber')
    
    # Classifying tokens by parts of speech
    tagged_tokens = pos_tag(tokens)
    
    # Filtering tokens to remove stopwords but keep adverbs (tagged as 'RB') and conjunctions (tagged as 'CC' for coordinating, 'IN' for subordinating)
    filtered_tokens = [
        word for word, tag in tagged_tokens 
        if word.lower() not in stop_words 
        or tag.startswith('RB')  # Adverbs
        or tag in ('CC', 'IN')  # Conjunctions
    ]

    return filtered_tokens

##### Remoção de URL's

A remoção de links é uma etapa de pré-processamento em tarefas de Processamento de Linguagem Natural (PLN) que envolve a eliminação de URLs ou links de um texto. Isso é feito para reduzir o ruído nos dados e focar nas palavras e frases que carregam mais significado.

Links geralmente não contribuem para a semântica de um texto e podem ser bastante variados e únicos, o que pode adicionar ruído e aumentar a dimensionalidade dos dados. Além disso, os links podem levar a conteúdo externo que está fora do contexto do texto atual, tornando a análise mais complexa.

In [ ]:
# Definir a função para remover URLs
def remove_urls(text):
    """
    Remove URLs de um texto fornecido.

    Args:
    texto (list): O texto de entrada contendo URLs.

    Returns:
    list: Lista de tokens com URLs removidas.
    """
    return re.sub(r'http\S+|www.\S+', '', text, flags=re.MULTILINE)

##### Correção de texto

A etapa de correção de texto no processamento de linguagem natural (NLP) envolve a identificação e correção de erros gramaticais em um texto. Isso inclui a correção de erros de concordância verbal e nominal, uso incorreto de tempos verbais, pontuação inadequada e outros aspectos gramaticais. Ferramentas de correção gramatical utilizam algoritmos e modelos linguísticos para analisar a estrutura das sentenças e sugerir melhorias, garantindo que o texto final seja gramaticalmente correto e claro. Esta etapa é crucial para melhorar a qualidade e a legibilidade do texto, especialmente em aplicações como assistentes de escrita, corretores ortográficos e sistemas de tradução automática.

In [ ]:
def spell_checker(words):
    """
    Corrige a ortografia de uma lista de palavras usando pyspellchecker,
    com exceções para palavras específicas. Garante que nenhum valor None seja retornado.

    Args:
        words (list of str): Lista de palavras a serem corrigidas.

    Returns:
        str: Texto com a ortografia corrigida.
    """
    spell = SpellChecker()
    exceptions = {'uber'}
    corrected_words = []

    for word in words:
        # Asegurar que a palavra não é None
        if word is None:
            corrected_words.append('')
        elif word.lower() in exceptions:
            corrected_words.append(word)
        else:
            # Obter a melhor sugestão de correção
            corrected_word = spell.correction(word) if word else ''
            corrected_words.append(corrected_word if corrected_word else '')

    # Assegurar que todos os elementos sejam strings
    corrected_words = [w if w is not None else '' for w in corrected_words]
    corrected_text = ' '.join(corrected_words)
    return corrected_text

##### Contrações

A etapa de correção de contrações envolve a expansão de palavras contraídas em suas formas completas. Por exemplo, transformar "can't" em "cannot" ou "it's" em "it is". Essa etapa é importante no processamento de linguagem natural (NLP) porque as contrações podem introduzir ambiguidades e dificuldades na análise de texto. Ao expandir as contrações, o texto se torna mais claro e consistente para os algoritmos, melhorando a precisão em tarefas como análise de sentimentos, tradução automática e extração de informações.

In [ ]:
def expand_contractions(text):
    """
    Expande contrações em um texto dado com base em um mapeamento fornecido.

    Args:
        text (str): Texto que contém contrações.
        contraction_mapping (dict): Um dicionário mapeando contrações para suas formas expandidas.

    Returns:
        str: Texto com todas as contrações expandidas de acordo com o mapeamento.
    """
    contraction_map = {
    "can't": "can not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't've": "cannot have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "could've": "could have",
    "could've": "could have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "i'm": "i am",
    "you're": "you are",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "we're": "we are",
    "they're": "they are",
    "i've": "i have",
    "you've": "you have",
    "we've": "we have",
    "they've": "they have",
    "i'd": "i would",
    "you'd": "you would",
    "he'd": "he would",
    "she'd": "she would",
    "we'd": "we would",
    "they'd": "they would",
    "i'll": "i will",
    "you'll": "you will",
    "he'll": "he will",
    "she'll": "she will",
    "we'll": "we will",
    "they'll": "they will",
}
    
    for contraction, expansion in contraction_map.items():
        text = text.replace(contraction, expansion)
    return text

# Exemplo de uso
sample_text = "I can't do this anymore, because it's too hard."
expanded_text = expand_contractions(sample_text)
print(expanded_text)

##### Balanceamento dos dados

A etapa de balanceamento das classes de dados é uma técnica usada em aprendizado de máquina e processamento de linguagem natural (NLP) para lidar com conjuntos de dados desbalanceados, onde uma ou mais classes têm significativamente mais exemplos do que outras. Esse desbalanceamento pode prejudicar o desempenho dos modelos, tornando-os tendenciosos em favor das classes majoritárias. Para balancear as classes, algumas estratégias comuns incluem subamostragem, que reduz o número de exemplos na classe majoritária (como remover 20% dos dados da classe negativa), e superamostragem, que aumenta o número de exemplos na classe minoritária (como duplicar os dados da classe positiva). Outra técnica é a geração de exemplos sintéticos, criando novos exemplos para a classe minoritária usando métodos como SMOTE (Synthetic Minority Over-sampling Technique). Essas técnicas ajudam a garantir que o modelo de aprendizado de máquina receba uma representação mais equilibrada de todas as classes, melhorando sua capacidade de generalizar e performar de maneira mais justa e precisa em dados não vistos.

In [ ]:
def remove_outliers_balance_negatives_and_multiply_positives(df):
    """
    Remove outliers do comprimento dos comentários para a classe de sentimentos negativos (-1),
    baseando-se nos quartis da própria classe negativa, e remove aleatoriamente 40% dos dados negativos
    restantes para ajudar no balanceamento das classes. Multiplica os dados da classe positiva (1) por 2,5 vezes
    para aumentar sua representatividade no dataset.

    Args:
        df (DataFrame): DataFrame contendo as colunas 'sentiment', 'comment', e 'comment_length'.

    Returns:
        DataFrame: DataFrame com outliers removidos da classe negativa, redução estratégica de 40% dos negativos,
                    e multiplicação dos dados positivos por 2,5 vezes.
    """
    # Adiciona a coluna 'comment_length' ao DataFrame se não existir
    if 'comment_length' not in df.columns:
        df['comment_length'] = df['comment'].apply(len)

    # Adiciona a coluna 'word_count'
    df['word_count'] = df['comment'].apply(lambda x: len(x.split()))
    
    # Filtra os dados para a classe negativa (-1)
    negativos = df[df['sentiment'] == -1]
    
    # Calcula os quartis apenas para a classe negativa
    Q1 = negativos['comment_length'].quantile(0.25)
    Q3 = negativos['comment_length'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR

    # Filtra os outliers na classe negativa baseado no limite calculado
    negativos_filtrados = negativos[negativos['comment_length'] <= upper_bound]
    
    # Amostragem aleatória para remover 20% dos dados negativos filtrados
    negativos_reduzidos = negativos_filtrados.sample(frac=0.83, random_state=42)

    # Multiplica os dados da classe positiva (1) por 2,5 vezes
    positivos = df[df['sentiment'] == 1]
    positivos_multiplicados = pd.concat([positivos] * 2 + [positivos.sample(frac=0.5, random_state=42)], ignore_index=True)
    
    # Combina os dados reduzidos de sentimentos negativos com as outras classes
    df_final = pd.concat([negativos_reduzidos, df[df['sentiment'] == 0], positivos_multiplicados], ignore_index=True)

    return df_final

##### Criação de nova coluna 

A coluna words_sentiment é adicionada ao DataFrame para indicar o sentimento de cada comentário com base na presença de palavras positivas ou negativas. O valor da coluna é determinado da seguinte maneira: 1 se o comentário contiver qualquer palavra positiva, -1 se o comentário contiver qualquer palavra negativa, e 0 se o comentário não contiver nem palavras positivas nem negativas.

In [ ]:
def has_sentiment_words(df):
    """
    Verifica a presença de palavras positivas e negativas em cada comentário de um DataFrame.

    Args:
    - df: DataFrame contendo os comentários a serem analisados.

    Returns:
    Um novo DataFrame com uma coluna 'words_sentiment', onde cada entrada indica o sentimento do comentário:
    - 1 se houver palavras positivas no comentário.
    - -1 se houver palavras negativas no comentário.
    - 0 se não houver nem palavras positivas nem negativas no comentário.
    """
    # Lista de palavras negativas e positivas
    negative_words = [
        'bad', 'terrible', 'awful', 'hate', 'not', 'hard', 'lose',
    'disaster', 'worst', 'poor', 'negative'
    ]
    positive_words = [
        'good', 'great', 'excellent', 'love', 'awesome', 'amazing',
    'best', 'positive', 'nice', 'happy'
    ]  
   
    
    # Função para verificar a presença de palavras negativas e positivas em um comentário
    def check_sentiment_words(text):
        """
        Verifica se um texto contém palavras com conotação negativa ou positiva.

        Args:
            text (str): O texto a ser analisado.

        Returns:
            int: Um valor inteiro indicando o sentimento do texto. Retorna -1 se o texto contém apenas palavras negativas,
                1 se contém apenas palavras positivas, e 0 se não contém nem palavras negativas nem positivas.
        """

        # Verifica se há palavras negativas
        has_negative = any(word in text for word in negative_words)
        # Verifica se há palavras positivas
        has_positive = any(word in text for word in positive_words)
        
        # Retorna -1 se negativas, 1 se positivas, e 0 se não houver nem negativas nem positivas
        if has_negative and not has_positive:
            return -1
        elif has_positive and not has_negative:
            return 1
        else:
            return 0
    
    # Aplicar a função a cada comentário e atribuir o resultado a uma nova coluna
    df['words_sentiment'] = df['comment'].apply(check_sentiment_words)
    
    return df

##### Pipeline

In [ ]:
# Definir o pipeline para pré-processamento dos comentários
pipeline = Pipeline([
    ('expand_contractions', FunctionTransformer(lambda x: x.apply(expand_contractions))),
    ('url_remover', FunctionTransformer(lambda x: x.apply(remove_urls))),
    ('tokenizer', FunctionTransformer(lambda x: x.apply(tokenize_text))),
    ('lemmatizacao', FunctionTransformer(lambda x: x.apply(lemmatize_tokens_with_pos))),
    ('punctuation_remover', FunctionTransformer(lambda x: x.apply(remove_punctuation_from_tokens))),
    ('number_remover', FunctionTransformer(lambda x: x.apply(remove_numbers_from_tokens))),
    ('stopwords_remover', FunctionTransformer(lambda x: x.apply(remove_stopwords_preserve_adverbs_conjunctions))),
    ('filter_empty_tokens', FunctionTransformer(lambda x: x.apply(filter_empty_tokens))),
    ('spell_checker', FunctionTransformer(lambda x: x.apply(spell_checker))),
])

# Primeiro aplicar a remoção de outliers e a redução de dados negativos
df = remove_outliers_balance_negatives_and_multiply_positives(df)
# Aplicar o pipeline ao DataFrame existente
df['comment'] = pipeline.fit_transform(df['comment'])
print(df)

##### Exportar dados pré-processados

In [ ]:
output_file_path = 'dados_pre_processados.csv'
df.to_csv(output_file_path, index=False)

print(f"Arquivo CSV exportado para {output_file_path}")

### Modelo Machine Learning

Nesta etapa, é construído e treinado o modelo de machine learning usando os dados vetorizados. O modelo pode ser de diferentes tipos, como regressão logística, máquinas de vetor de suporte (SVM), redes neurais, entre outros. O processo inclui a escolha do algoritmo, o ajuste dos hiperparâmetros, o treinamento com dados de treino e a validação com dados de teste para avaliar o desempenho do modelo com base em métricas como accuracy, precision, recall e F1-score.

##### Stack

In [ ]:
current_dir = os.getcwd()
csv_path = os.path.join(current_dir, 'dados_pre_processados.csv')
df = pd.read_csv(csv_path)
df.head()
df.drop("words_sentiment", axis=1, inplace=True)
df['comment'] = df['comment'].apply(str)

# Função para vetorização usando bigramas
def sklearn_bow_bigrams(docs):
    """
    Cria uma matriz de termos do documento usando a representação Bag of Words com bigramas (sequências de dois termos consecutivos).

    Args:
        docs (list): Uma lista de documentos (strings) para os quais a matriz Bag of Words deve ser criada.

    Returns:
        Uma tupla contendo a matriz Bag of Words e o objeto CountVectorizer treinado.

    """
    vectorizer = CountVectorizer(ngram_range=(1, 4))
    bow_matrix = vectorizer.fit_transform(docs)
    return bow_matrix, vectorizer

# Gerar matriz BoW usando a coluna 'comment'
X, vectorizer = sklearn_bow_bigrams(df['comment'])

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.15, random_state=42)

In [ ]:

# Definindo os estimadores com os melhores parâmetros encontrados
best_estimators = [
    ('rf', RandomForestClassifier(n_estimators=150, max_depth=20, random_state=42)),
    ('et', ExtraTreesClassifier(n_estimators=150, max_depth=None, random_state=42)), 
    ('logreg', LogisticRegression())  # O parâmetro C é usado apenas no meta-modelo
]

# Definindo o meta-modelo com o melhor parâmetro C encontrado
final_estimator = LogisticRegression(C=10000.0)

# Definindo o modelo de stacking com os melhores parâmetros
best_stack_model = StackingClassifier(
    estimators=best_estimators,
    final_estimator=final_estimator,
    stack_method='auto'
)

# Treinamento do modelo de stacking com os melhores parâmetros
best_stack_model.fit(X_train, y_train)

In [ ]:
# Importar a função cohen_kappa_score
from sklearn.metrics import cohen_kappa_score

# Prever os sentimentos no conjunto de teste
y_pred = best_stack_model.predict(X_test)

# Calcular a precisão
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Gerar e imprimir o relatório de classificação
print(classification_report(y_test, y_pred))

# Calcular o Kappa de Cohen
kappa = cohen_kappa_score(y_test, y_pred)
print("Cohen's Kappa:", kappa)

# Rótulos reais usados no dataset
labels = [-1, 0, 1]

# Criar a matriz de confusão com rótulos específicos
conf_matrix = confusion_matrix(y_test, y_pred, labels=labels)

# Plotar a matriz de confusão usando Seaborn
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', cbar=False, xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


### Exportação do modelo

Após a construção e validação, o modelo final é exportado para ser utilizado em um ambiente de produção. Isso envolve salvar o modelo treinado em um formato que pode ser facilmente carregado e executado em aplicações externas, no caso em nossa API. A exportação garante que o modelo possa ser reutilizado sem a necessidade de ser re-treinado, facilitando a sua integração em sistemas de classificação de sentimentos em tempo real.

In [ ]:
def pipeline_test(text):
    """
    Aplica as etapas de pré-processamento de texto a um texto de entrada.

    Args:
        text (str): O texto de entrada a ser pré-processado.

    Returns:
        list: Uma lista de tokens após o pré-processamento.
    """
    text = remove_urls(text)
    tokens = tokenize_text(text)
    tokens = lemmatize_tokens_with_pos(tokens)
    tokens = remove_punctuation_from_tokens(tokens)
    tokens = remove_numbers_from_tokens(tokens)
    tokens = remove_stopwords_preserve_adverbs_conjunctions(tokens)
    tokens = filter_empty_tokens(tokens)
    tokens = spell_checker(tokens)
    return tokens

# Exemplo de uso
sample_text = "Unsafe"
pipeline_test(sample_text)

In [ ]:
# Salvar o modelo e o vetorizador em um arquivo pickle
with open('stack_model.pkl', 'wb') as file:
    pickle.dump(best_stack_model, file)

with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)


In [ ]:
modelo_path = 'stack_model.pkl'
vectorizer_path = 'vectorizer.pkl'

with open(modelo_path, 'rb') as file:
    loaded_model = pickle.load(file)

with open(vectorizer_path, 'rb') as file:
    loaded_vectorizer = pickle.load(file)

def preprocessar_texto(texto):
    """
    Pré-processa o texto usando uma pipeline de processamento definida.

    Args:
    texto (str): Texto a ser processado.

    Returns:
    str: Texto processado.
    """
    # Supondo que 'pipeline_test' seja uma função definida em outro lugar que processa o texto
    return pipeline_test(texto)

def fazer_previsao(comentario):
    """
    Faz previsão de um comentário usando o modelo e o vetorizador carregados.

    Args:
    comentario (str): Comentário para o qual se deseja fazer a previsão.

    Returns:
    str: Previsão do modelo para o comentário.
    """
    texto_processado = preprocessar_texto(comentario)
    comentario_vetorizado = loaded_vectorizer.transform([texto_processado])
    previsao = loaded_model.predict(comentario_vetorizado)
    return previsao[0]
